# Analyzing Orange Telecoms Customer Churn Dataset

Predicting which customer is likely to cancel subscription to a service. It has become a common practise across banks, ISPs, insurance firms and credit card companies.

### Run `Apache pyspark` from the terminal:
Load `pyspark` with the [Spark-CSV](http://spark-packages.org/package/databricks/spark-csv) package.

> IPYTHON_OPTS="notebook" ~/path_to_pyspark --packages com.databricks:spark-csv_2.11:1.4.0

In [1]:
import pandas as pd
import plotly.plotly as py
import cufflinks as cf

## Fetching and Importing Churn Dataset
For this tutorial, we'll be using the Orange Telecoms Churn Dataset. It consists of cleaned customer activity data (features), along with a churn label specifying whether the customer canceled their subscription or not. The data can be fetched from BigML's S3 bucket, [churn-80](https://bml-data.s3.amazonaws.com/churn-bigml-80.csv) and [churn-20](https://bml-data.s3.amazonaws.com/churn-bigml-20.csv). The two sets are from the same batch, but have been split by an 80/20 ratio. We'll use the larger set for training and cross-validation purposes, and the smaller set for final testing and model performance evaluation. The two data sets have been included in this repository for convenience.

In [3]:
# Import dataset with spark CSV package
orange_sprk_df = sqlContext.read.load("../_datasets_downloads/churn-bigml-80.csv",
                                format='com.databricks.spark.csv',
                                header='true',
                                inferschema='true')

orange_final_dataset = sqlContext.read.load("../_datasets_downloads/churn-bigml-20.csv",
                                format='com.databricks.spark.csv',
                                header='true',
                                inferschema='true')

# Print Dataframe Schema. That's DataFrame = Dataset[Row]
orange_sprk_df.cache()
orange_sprk_df.printSchema()

root
 |-- State: string (nullable = true)
 |-- Account length: integer (nullable = true)
 |-- Area code: integer (nullable = true)
 |-- International plan: string (nullable = true)
 |-- Voice mail plan: string (nullable = true)
 |-- Number vmail messages: integer (nullable = true)
 |-- Total day minutes: double (nullable = true)
 |-- Total day calls: integer (nullable = true)
 |-- Total day charge: double (nullable = true)
 |-- Total eve minutes: double (nullable = true)
 |-- Total eve calls: integer (nullable = true)
 |-- Total eve charge: double (nullable = true)
 |-- Total night minutes: double (nullable = true)
 |-- Total night calls: integer (nullable = true)
 |-- Total night charge: double (nullable = true)
 |-- Total intl minutes: double (nullable = true)
 |-- Total intl calls: integer (nullable = true)
 |-- Total intl charge: double (nullable = true)
 |-- Customer service calls: integer (nullable = true)
 |-- Churn: boolean (nullable = true)



In [8]:
# Display first 5 Rows or Spark Dataset
orange_sprk_df.toPandas().head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


## Summary Statistics
Show the summary statistic for the dataset. The describe function compute statistics ONLY for columns having numeric data types. So we can extract the numeric indexes from the resulting pandas dataframe.

In [52]:
num_set = orange_sprk_df.describe().toPandas().transpose()
num_set.head()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Account length,2666,100.62040510127532,39.56397365334986,1,243
Area code,2666,437.43885971492875,42.52101801942723,408,510
Number vmail messages,2666,8.021755438859715,13.612277018291945,0,50
Total day minutes,2666,179.48162040510107,54.21035022086984,0.0,350.8


In [45]:
# Display the numeric index
num_set.index.values

array(['summary', 'Account length', 'Area code', 'Number vmail messages',
       'Total day minutes', 'Total day calls', 'Total day charge',
       'Total eve minutes', 'Total eve calls', 'Total eve charge',
       'Total night minutes', 'Total night calls', 'Total night charge',
       'Total intl minutes', 'Total intl calls', 'Total intl charge',
       'Customer service calls'], dtype=object)

In [47]:
# Drop the summary column and slice the dataframe using the numeric index.
new_df = orange_sprk_df.toPandas()
new_df = new_df[num_set.index.drop('summary')]
new_df.head()

,Account length,Area code,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
0,128,415,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


Index([u'Account length', u'Area code', u'Number vmail messages',
       u'Total day minutes', u'Total day calls', u'Total day charge',
       u'Total eve minutes', u'Total eve calls', u'Total eve charge',
       u'Total night minutes', u'Total night calls', u'Total night charge',
       u'Total intl minutes', u'Total intl calls', u'Total intl charge',
       u'Customer service calls'],
      dtype='object')